BIBLIOTECAS

In [1]:
import numpy as np
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import train_test_split
from sklearn.datasets import make_classification
from sklearn.metrics import accuracy_score

Funções de Ativação e Suas Derivadas

In [2]:
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

def sigmoid_derivative(x):
    return x * (1 - x)

def relu(x):
    return np.maximum(0, x)

def relu_derivative(x):
    return (x > 0).astype(float)

def tanh(x):
    return np.tanh(x)

def tanh_derivative(x):
    return 1 - (x ** 2)

def linear(x): # Função de ativação linear (identidade)
    return x

def linear_derivative(x):
    return np.ones_like(x)

# Dicionário para mapear strings para funções
ACTIVATION_FUNCTIONS = {
    'sigmoid': {'func': sigmoid, 'derivative': sigmoid_derivative},
    'relu': {'func': relu, 'derivative': relu_derivative},
    'tanh': {'func': tanh, 'derivative': tanh_derivative},
    'linear': {'func': linear, 'derivative': linear_derivative}
}

Implementação da Classe MLP

In [3]:
class MyMLP:
    def __init__(self, layer_dims, activation_funcs, learning_rate=0.01, epochs=1000):
        """
        Inicializa o Multi Layer Perceptron.

        Args:
            layer_dims (list): Uma lista de inteiros
            activation_funcs (list): Uma lista de strings com os nomes das funções de ativação
                                     para cada camada oculta e a camada de saída.
                                     O tamanho deve ser layer_dims - 1
            learning_rate (float): A taxa de aprendizado para o otimizador.
            epochs (int): O número de épocas de treinamento.
        """
        self.layer_dims = layer_dims
        self.learning_rate = learning_rate
        self.epochs = epochs
        self.weights = []
        self.biases = []
        self.activation_funcs = []

        # Validação das funções de ativação
        if len(activation_funcs) != len(layer_dims) - 1:
            raise ValueError("O número de funções de ativação deve ser igual ao número de camadas - 1.")

        for af_name in activation_funcs:
            if af_name not in ACTIVATION_FUNCTIONS:
                raise ValueError(f"Função de ativação '{af_name}' não suportada.")
            self.activation_funcs.append(ACTIVATION_FUNCTIONS[af_name])

        # Inicializa pesos e bias
        for i in range(len(self.layer_dims) - 1):
            weight_matrix = np.random.randn(self.layer_dims[i], self.layer_dims[i+1]) * 0.01
            bias_vector = np.zeros((1, self.layer_dims[i+1]))
            self.weights.append(weight_matrix)
            self.biases.append(bias_vector)

    def _forward_pass(self, X):
        """
        Realiza a propagação direta (forward pass).
        Retorna as ativações de todas as camadas para uso no backpropagation.
        """
        activations = [X]
        current_activation = X
        for i in range(len(self.weights)):
            # Z = WX + B
            z = np.dot(current_activation, self.weights[i]) + self.biases[i]
            # A = g(Z)
            current_activation = self.activation_funcs[i]['func'](z)
            activations.append(current_activation)
        return activations

    def _backward_pass(self, X, y, activations):
        """
        Realiza a propagação reversa (backward pass) e atualiza pesos e bias.
        """
        num_layers = len(self.layer_dims)
        deltas = [None] * num_layers
        d_weights = [None] * (num_layers - 1)
        d_biases = [None] * (num_layers - 1)

        # Erro na camada de saída
        # dE/dA_L = A_L - Y (para MSE)
        error_output = activations[-1] - y

        # Delta da camada de saída
        # delta_L = dE/dA_L * g'(Z_L)
        deltas[-1] = error_output * self.activation_funcs[-1]['derivative'](activations[-1])

        # Propagação do erro para trás
        for i in reversed(range(num_layers - 1)):
            # dW = A_{L-1}^T * delta_L
            d_weights[i] = np.dot(activations[i].T, deltas[i+1])
            # db = sum(delta_L)
            d_biases[i] = np.sum(deltas[i+1], axis=0, keepdims=True)

            if i > 0: # Não calcula delta para a camada de entrada
                # delta_L-1 = delta_L * W_L^T * g'(Z_L-1)
                deltas[i] = np.dot(deltas[i+1], self.weights[i].T) * self.activation_funcs[i-1]['derivative'](activations[i])

        # Atualiza pesos e bias
        for i in range(num_layers - 1):
            self.weights[i] -= self.learning_rate * d_weights[i]
            self.biases[i] -= self.learning_rate * d_biases[i]

    def train(self, X, y):
        """
        Treina o MLP usando o algoritmo de backpropagation.

        Args:
            X (np.array): Dados de entrada de treinamento.
            y (np.array): Rótulos verdadeiros.
        """
        for epoch in range(self.epochs):
            activations = self._forward_pass(X)
            self._backward_pass(X, y, activations)

            if epoch % (self.epochs // 10) == 0:
                loss = np.mean(np.square(y - activations[-1])) # Mean Squared Error
                print(f"Epoch {epoch}/{self.epochs}, Loss: {loss:.4f}")

    def predict(self, X):
        """
        Faz previsões com o MLP treinado.
        """
        activations = self._forward_pass(X)
        # Para classificação, geralmente usamos um limiar de 0.5 para sigmoid
        # ou argmax para softmax (se você adicionar softmax na saída)
        return (activations[-1] > 0.5).astype(int) # para classificação binária com sigmoid na saída

Teste e Comparação

In [4]:
# Gerar dados de exemplo
X, y = make_classification(n_samples=1000, n_features=20, n_informative=10,
                           n_redundant=5, n_classes=2, random_state=42)
y = y.reshape(-1, 1) # Scikit-learn espera 1D, mas nossa implementação espera 2D

# Dividir em conjuntos de treinamento e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print(f"Forma dos dados de treinamento X: {X_train.shape}, y: {y_train.shape}")
print(f"Forma dos dados de teste X: {X_test.shape}, y: {y_test.shape}")

# --- Teste com a sua implementação ---
print("\n--- Treinando meu MLP ---")
# Exemplo: 20 -> 10 -> 5 -> 1 (entrada -> oculta1 -> oculta2 -> saída)
# Funções de ativação para as camadas ocultas e de saída
my_mlp = MyMLP(layer_dims=[X_train.shape[1], 10, 5, y_train.shape[1]],
               activation_funcs=['relu', 'tanh', 'sigmoid'], # relu para 10, tanh para 5, sigmoid para 1
               learning_rate=0.01,
               epochs=2000)
my_mlp.train(X_train, y_train)

y_pred_my_mlp = my_mlp.predict(X_test)
accuracy_my_mlp = accuracy_score(y_test, y_pred_my_mlp)
print(f"\nAcurácia do meu MLP: {accuracy_my_mlp:.4f}")

# --- Teste com Scikit-learn ---
print("\n--- Treinando MLP do Scikit-learn ---")
# Scikit-learn espera y como 1D para classificação binária
y_train_sklearn = y_train.flatten()
y_test_sklearn = y_test.flatten()

# Note que a configuração do Scikit-learn é um pouco diferente
# hidden_layer_sizes define as camadas ocultas.
# 'logistic' é sigmoid, 'relu' é ReLU, 'tanh' é tanh.
mlp_sklearn = MLPClassifier(hidden_layer_sizes=(10, 5), # Duas camadas ocultas com 10 e 5 neurônios
                            activation='relu', # Ativação para as camadas ocultas
                            solver='adam', # Otimizador Adam, mais robusto que SGD simples
                            alpha=0.0001, # Regularização L2
                            batch_size='auto',
                            learning_rate_init=0.01,
                            max_iter=2000,
                            random_state=42,
                            verbose=False) # Mude para True para ver o treinamento

mlp_sklearn.fit(X_train, y_train_sklearn)

y_pred_sklearn = mlp_sklearn.predict(X_test)
accuracy_sklearn = accuracy_score(y_test_sklearn, y_pred_sklearn)
print(f"Acurácia do MLP do Scikit-learn: {accuracy_sklearn:.4f}")

Forma dos dados de treinamento X: (800, 20), y: (800, 1)
Forma dos dados de teste X: (200, 20), y: (200, 1)

--- Treinando meu MLP ---
Epoch 0/2000, Loss: 0.2500
Epoch 200/2000, Loss: 0.0190
Epoch 400/2000, Loss: 0.0090
Epoch 600/2000, Loss: 0.0088
Epoch 800/2000, Loss: 0.0088
Epoch 1000/2000, Loss: 0.0087
Epoch 1200/2000, Loss: 0.0075
Epoch 1400/2000, Loss: 0.0075
Epoch 1600/2000, Loss: 0.0075
Epoch 1800/2000, Loss: 0.0075

Acurácia do meu MLP: 0.9200

--- Treinando MLP do Scikit-learn ---
Acurácia do MLP do Scikit-learn: 0.9150
